https://docs.google.com/forms/d/e/1FAIpQLSdI6cWyyHCBkn2h0lUXvZM9iGNX3y1QMRbKT0iSVsrm8Qhx_w/viewform?hr_submission=ChcIrpDc8gMSDwi72JqF0wgSBgiGjdbHJxAB

In [1]:
import pandas as pd
import numpy as np

import urllib.request
import io
import time
import copy


from scipy.io import arff
from sklearn.model_selection import KFold
from collections import Counter

from sklearn.preprocessing import MinMaxScaler
import re
import random

import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning) 

# Preparação de dados

* extração do dataframe

In [2]:
url = 'http://promise.site.uottawa.ca/SERepository/datasets/kc1.arff'
ftpstream = urllib.request.urlopen(url)
data, meta = arff.loadarff(io.StringIO(ftpstream.read().decode('utf-8')))
kc1 = pd.DataFrame(data)
kc1 = kc1.sample(frac=1, random_state=20)
kc1.reset_index(drop=True, inplace=True)

# Algoritmos

In [3]:
def  euclidian_distance(x1, x2):
    arr = []
    for i in range(len(x1)-1):
        arr.append((x1[i] - x2[i])** 2)
    return np.sqrt(np.sum(arr))


In [4]:
class KNN:

    def __init__(self, k):
        self.k = k

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y


    def predict(self, X):
        predicted_labels = []
        for _, row in X.iterrows():
            predicted_labels.append(self.predict_func(row))

        return np.array(predicted_labels)
        
    def predict_func(self, x):
        distances = []
        for _, row in self.X_train.iterrows():
            distances.append(euclidian_distance(x, row))
          
        k_indices = np.argsort(distances)[:self.k]        
        k_nearest_labels = [self.y_train.iloc[i] for i in k_indices]

        unique, counts = np.unique(k_nearest_labels, return_counts=True)
        predicted = unique[counts.argmax()]

        return predicted

In [5]:
def attr_class(df):
    Y = df["defects"]
    X = df.drop(columns=["defects"]) 
    
    scaler = MinMaxScaler(feature_range=(0, 1))
    x_scaled = scaler.fit_transform(X)
    X = pd.DataFrame(x_scaled)
    
    Y = Y.apply(str).str.replace("b|'", '')

    
    return X, Y

In [6]:
def _attr_class(df):
    
    x = []
    y = []
    for row in df:
        x.append(row[:-1])
        y.append(row[-1])

    
    return x,y

### LVQ 1

In [7]:
from sklearn.neighbors import KNeighborsClassifier


In [8]:
class LVQ1:   
    
    def __init__(self, n_prototypes):
        self.n_prototypes  = n_prototypes
        self._epochs        = 10
        self._lrate        = 0.25

    
    def random_prototype(self, train):
        n_records  = train.shape[0]
        n_features = train.shape[1]
        prototypes  = []
        records    = []
        random.seed(10)
        for i in range(self.n_prototypes):
            records.append(random.randint(0, n_records-1))
        
        for r in records:
            prototype = [train.iloc[r][i] for i in range(n_features)]
            prototypes.append(prototype)
        
        return prototypes, records
    

    def trainPrototypes(self,train):
        prototypes = []
        records = []
        aux = train
        for i in range(self.n_prototypes):
            prototypes, records = self.random_prototype(train)
        for i in records:
            aux = aux.drop(aux.loc[aux.index==i].index)
        aux = aux.reset_index(drop=True)
        
        x_train, y_train = attr_class(aux)
        x_proto, y_proto = _attr_class(prototypes)

        knn = KNeighborsClassifier(n_neighbors=1)
        knn.fit(x_train, y_train)
        for epoch in range(self._epochs):            
            rate  = self._lrate * (1.0 - (epoch / float(self._epochs)))
            predicted_classes = knn.predict(x_proto) 
            real_instances = knn.kneighbors(X=x_proto, n_neighbors=1, return_distance=False)
            for idx in range(len(x_proto)):
                prototype = x_proto[idx]
                proto_class = y_proto[idx]
                
                predicted_class = predicted_classes[idx] 
                real_instance_index = real_instances[idx][0]
                real_instance = x_train.iloc[real_instance_index] 
                
                for p in range(len(prototype)): 
                    proto_val = prototype[p]
                    real_val  = real_instance[p]
                    error = rate * (real_val - proto_val)
                    if proto_class == predicted_class:
                        prototype[p] += error
                    else:
                        prototype[p] -= error

       
        return pd.DataFrame(prototypes, columns = train.columns)

lvq1 = LVQ1(10)
df  = lvq1.trainPrototypes(kc1)
df 

,loc,v(g),ev(g),iv(g),n,v,l,d,i,e,...,lOCode,lOComment,lOBlank,locCodeAndComment,uniq_Op,uniq_Opnd,total_Op,total_Opnd,branchCount,defects
0,11.0,2.0,1.0,2.0,23.0,85.11,0.16,6.42,13.26,546.12,...,8.0,0.0,1.0,0.0,7.0,6.0,12.0,11.0,3.0,b'false'
1,13.0,2.0,1.0,2.0,26.0,98.99,0.12,8.10,12.22,801.83,...,10.0,0.0,1.0,0.0,9.0,5.0,17.0,9.0,3.0,b'false'
2,38.0,5.0,1.0,5.0,61.0,307.71,0.07,15.00,20.51,4615.62,...,19.0,0.0,17.0,0.0,18.0,15.0,36.0,25.0,7.0,b'true'
3,4.0,1.0,1.0,1.0,4.0,8.00,0.67,1.50,5.33,12.00,...,0.0,0.0,0.0,0.0,3.0,1.0,3.0,1.0,1.0,b'false'
4,17.0,2.0,1.0,2.0,25.0,92.51,0.16,6.40,14.45,592.07,...,12.0,0.0,3.0,0.0,8.0,5.0,17.0,8.0,3.0,b'false'
5,9.0,1.0,1.0,1.0,9.0,27.00,0.40,2.50,10.80,67.50,...,4.0,1.0,2.0,0.0,5.0,3.0,6.0,3.0,1.0,b'false'
6,2.0,1.0,1.0,1.0,4.0,8.00,0.67,1.50,5.33,12.00,...,0.0,0.0,0.0,0.0,3.0,1.0,3.0,1.0,1.0,b'false'
7,20.0,1.0,1.0,1.0,62.0,291.43,0.18,5.56,52.46,1619.04,...,12.0,0.0,0.0,0.0,8.0,18.0,37.0,25.0,1.0,b'false'
8,4.0,1.0,1.0,1.0,8.0,24.00,0.29,3.50,6.86,84.00,...,2.0,0.0,0.0,0.0,7.0,1.0,7.0,1.0,1.0,b'false'
9,9.0,1.0,1.0,1.0,18.0,68.53,0.21,4.67,14.69,319.82,...,6.0,0.0,0.0,0.0,8.0,6.0,11.0,7.0,1.0,b'false'


### LVQ 2.1

In [9]:
class LVQ2_1:   
    
    def __init__(self, n_prototypes, window):
        self.n_prototypes   = n_prototypes
        self._epochs        = 100
        self._lrate         = 0.25
        self._window        = window

    
    def inside(self):
        return (1.0 - self._window) / (1.0 + self._window)
    
    def window(self, d1, d2):
        a  = d1/d2
        b  = d2/d1
        minimum = min(a,b)
        return minimum > self.inside()
    
    def updatePrototypes(self, prototype, real, same_class, rate):
        for idx in range(21):
            proto_val = prototype[idx]
            real_val  = real[idx]
            error = rate * (real_val - proto_val)
            if same_class:
                prototype[idx] += error
            else:
                prototype[idx] -= error
        return prototype
    
    def trainPrototypes(self,train, df_prototypes):
        random.seed(10)

        x_train, y_train = attr_class(train)
        x_proto, y_proto = attr_class(df_prototypes)
        knn = KNeighborsClassifier(n_neighbors=2)
        knn.fit(x_proto, y_proto)
        for epoch in range(self._epochs):
            rate  = self._lrate * (1.0 - (epoch / float(self._epochs)))
            idx = random.randrange(0, len(x_train))
            
            x_neighbors = knn.kneighbors(X=x_train, n_neighbors=2, return_distance=False)
            x_instance = x_train.iloc[idx]
            x_class = y_train.iloc[idx]

            #mais próximo
            n1_idx   = x_neighbors[idx][0]
            n1       = x_proto.iloc[n1_idx]
            n1       = n1[:21] 
            n1_class = y_proto[n1_idx]
            n1_dist  = euclidian_distance(n1, x_instance) 

            #segundo mais próximo
            n2_idx   = x_neighbors[idx][0]
            n2       = x_proto.iloc[n2_idx]
            n2       = n2[:21]
            n2_class = y_proto[n2_idx]
            n2_dist  = euclidian_distance(n2, x_instance)

            insideWindow = self.window(n1_dist, n2_dist)
            if insideWindow and n1_class != n2_class:
                x_proto.iloc[n1_idx] = self.updatePrototypes(n1, x_instance, n1_class == x_class, rate)
                x_proto.iloc[n2_idx] = self.updatePrototypes(n2, x_instance, n2_class == x_class, rate)
        prototypes = pd.concat([x_proto, y_proto], axis=1)
        prototypes.columns = kc1.columns
        return prototypes 
    
lvq2 = LVQ2_1(10, 0.2)
df2  = lvq2.trainPrototypes(kc1, df)
df2

,loc,v(g),ev(g),iv(g),n,v,l,d,i,e,...,lOCode,lOComment,lOBlank,locCodeAndComment,uniq_Op,uniq_Opnd,total_Op,total_Opnd,branchCount,defects
0,0.250000,0.25,0.0,0.25,0.327586,0.257282,0.150000,0.364444,0.168258,0.116022,...,0.421053,0.0,0.058824,0.0,0.266667,0.294118,0.264706,0.416667,0.333333,false
1,0.305556,0.25,0.0,0.25,0.379310,0.303593,0.083333,0.488889,0.146191,0.171567,...,0.526316,0.0,0.058824,0.0,0.400000,0.235294,0.411765,0.333333,0.333333,false
2,1.000000,1.00,0.0,1.00,0.982759,1.000000,0.000000,1.000000,0.322088,1.000000,...,1.000000,0.0,1.000000,0.0,1.000000,0.823529,0.970588,1.000000,1.000000,true
3,0.055556,0.00,0.0,0.00,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,false
4,0.416667,0.25,0.0,0.25,0.362069,0.281973,0.150000,0.362963,0.193507,0.126003,...,0.631579,0.0,0.176471,0.0,0.333333,0.235294,0.411765,0.291667,0.333333,false
5,0.194444,0.00,0.0,0.00,0.086207,0.063395,0.550000,0.074074,0.116062,0.012056,...,0.210526,1.0,0.117647,0.0,0.133333,0.117647,0.088235,0.083333,0.000000,false
6,0.000000,0.00,0.0,0.00,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,false
7,0.500000,0.00,0.0,0.00,1.000000,0.945681,0.183333,0.300741,1.000000,0.349082,...,0.631579,0.0,0.000000,0.0,0.333333,1.000000,1.000000,1.000000,0.000000,false
8,0.055556,0.00,0.0,0.00,0.068966,0.053385,0.366667,0.148148,0.032463,0.015640,...,0.105263,0.0,0.000000,0.0,0.266667,0.000000,0.117647,0.000000,0.000000,false
9,0.194444,0.00,0.0,0.00,0.241379,0.201962,0.233333,0.234815,0.198600,0.066865,...,0.315789,0.0,0.000000,0.0,0.333333,0.294118,0.235294,0.250000,0.000000,false


### LVQ 3

In [11]:
class LVQ3:   
    
    def __init__(self, n_prototypes, window):
        self.n_prototypes  = n_prototypes
        self._epochs       = 100
        self._lrate        = 0.25
        self._window       = window
        self._epsilon      = 0.75
        
    def getNeighborPrototype(self, prototypes, row):
        distances  = []
        final_dist = []
        for _,proto in prototypes.iterrows():
            dist = euclidian_distance(proto, row)
            distances.append((proto, dist))
        distances.sort(key=lambda tup: tup[1])
        final_dist.append(distances[1])
        final_dist.append(distances[2])
        return final_dist
    
    def inside(self):
        return (1.0 - self._window) / (1.0 + self._window)
    
    def window(self, d1, d2):
        a  = d1/d2
        b  = d2/d1
        minimum = min(a,b)
        return minimum > self.inside()
    
    def updatePrototypes(self, prototype, real, same_class, rate):
        for idx in range(21):
            proto_val = prototype[idx]
            real_val  = real[idx]
            error = rate * (real_val - proto_val)
            if same_class:
                prototype[idx] += error
            else:
                prototype[idx] -= error
        return prototype
    
    def trainPrototypes(self,train, df_prototypes):
        x_train, y_train = attr_class(train)
        x_proto, y_proto = attr_class(df_prototypes)
        knn = KNeighborsClassifier(n_neighbors=2)
        knn.fit(x_proto, y_proto)        
        for epoch in range(self._epochs):
            rate = self._lrate * (1.0 - (epoch / float(self._epochs)))
            x_neighbors = knn.kneighbors(X=x_train, n_neighbors=2, return_distance=False)
            for idx, _ in x_train.iterrows():
                x_instance = x_train.iloc[idx]
                x_class = y_train.iloc[idx]

                #mais próximo
                n1_idx   = x_neighbors[idx][0]
                n1       = x_proto.iloc[n1_idx]
                n1       = n1[:21] 
                n1_class = y_proto[n1_idx]
                n1_dist  = euclidian_distance(n1, x_instance) 

                #segundo mais próximo
                n2_idx   = x_neighbors[idx][0]
                n2       = x_proto.iloc[n2_idx]
                n2       = n2[:21]
                n2_class = y_proto[n2_idx]
                n2_dist  = euclidian_distance(n2, x_instance)

                insideWindow = self.window(n1_dist, n2_dist)
                if insideWindow:
                    if n1_class != n2_class:
                        x_proto.iloc[n1_idx] = self.updatePrototypes(n1, x_instance, n1_class == x_class, rate)
                        x_proto.iloc[n2_idx] = self.updatePrototypes(n2, x_instance, n2_class == x_class, rate)
                    else:
                        rate = rate * self._epsilon
                        x_proto.iloc[n1_idx] = self.updatePrototypes(n1, x_instance, True, rate)
                        x_proto.iloc[n2_idx] = self.updatePrototypes(n2, x_instance, True, rate)

        prototypes = pd.concat([x_proto, y_proto], axis=1)
        prototypes.columns = kc1.columns
        return prototypes
    
lvq3 = LVQ3(10, 0.25)
df3  = lvq3.trainPrototypes(kc1, df)
df3

,loc,v(g),ev(g),iv(g),n,v,l,d,i,e,...,lOCode,lOComment,lOBlank,locCodeAndComment,uniq_Op,uniq_Opnd,total_Op,total_Opnd,branchCount,defects
0,2.500000e-01,0.250000,9.728797e-14,0.250000,3.275862e-01,2.572820e-01,1.500000e-01,3.644444e-01,1.682580e-01,1.160217e-01,...,4.210526e-01,5.527727e-14,5.882353e-02,1.933573e-180,2.666667e-01,2.941176e-01,2.647059e-01,4.166667e-01,0.333333,false
1,2.589864e-01,0.211290,2.043868e-01,0.191935,2.464731e-01,1.979172e-01,2.088158e-02,7.077279e-01,1.781127e-01,1.748580e-01,...,2.698742e-01,1.565812e-06,1.849236e-01,1.621349e-10,6.347685e-01,2.407216e-01,2.545346e-01,2.345090e-01,0.223655,false
2,1.000000e+00,1.000000,9.377898e-132,1.000000,9.827586e-01,1.000000e+00,3.907457e-133,1.000000e+00,3.220878e-01,1.000000e+00,...,1.000000e+00,7.992527e-132,1.000000e+00,1.564886e-162,1.000000e+00,8.235294e-01,9.705882e-01,1.000000e+00,1.000000,true
3,5.555556e-02,0.000000,0.000000e+00,0.000000,9.584032e-102,3.108172e-102,1.000000e+00,1.972082e-101,2.549783e-100,3.785652e-104,...,4.854934e-101,0.000000e+00,2.864711e-140,0.000000e+00,5.729697e-101,7.066626e-101,3.126826e-102,1.981297e-101,0.000000,false
4,4.166906e-01,0.250014,2.166211e-04,0.250014,3.620023e-01,2.819365e-01,1.499248e-01,3.629930e-01,1.936377e-01,1.259995e-01,...,6.314010e-01,3.282138e-04,1.765511e-01,6.491202e-25,3.335068e-01,2.354384e-01,4.116660e-01,2.916458e-01,0.333297,false
5,1.944444e-01,0.000000,0.000000e+00,0.000000,8.620690e-02,6.339461e-02,5.500000e-01,7.407407e-02,1.160620e-01,1.205573e-02,...,2.105263e-01,1.000000e+00,1.176471e-01,0.000000e+00,1.333333e-01,1.176471e-01,8.823529e-02,8.333333e-02,0.000000,false
6,2.045397e-30,0.000000,0.000000e+00,0.000000,3.980754e-31,8.803017e-32,1.000000e+00,1.365186e-30,7.229140e-30,1.075362e-33,...,1.680429e-30,0.000000e+00,0.000000e+00,0.000000e+00,3.966419e-30,2.445949e-30,2.164565e-31,6.857801e-31,0.000000,false
7,5.000000e-01,0.000000,0.000000e+00,0.000000,1.000000e+00,9.456808e-01,1.833333e-01,3.007407e-01,1.000000e+00,3.490818e-01,...,6.315789e-01,0.000000e+00,0.000000e+00,0.000000e+00,3.333333e-01,1.000000e+00,1.000000e+00,1.000000e+00,0.000000,false
8,5.424934e-02,0.012574,1.857487e-03,0.010655,3.270245e-02,2.040766e-02,1.805866e-01,8.439104e-02,1.780569e-01,2.786846e-03,...,4.331531e-02,6.870342e-03,2.065016e-02,1.959374e-06,1.860751e-01,9.752693e-02,3.153142e-02,3.455749e-02,0.012574,false
9,1.467625e-01,0.076920,7.853560e-02,0.076874,1.102322e-01,7.983699e-02,5.276481e-02,2.018807e-01,3.208363e-01,2.347220e-02,...,1.274253e-01,1.493513e-02,8.490761e-02,5.375738e-15,2.944393e-01,2.083582e-01,1.091357e-01,1.119691e-01,0.073687,false


## KNN 


In [12]:
def run(df, title):
    kf = KFold(n_splits=5)
    accuracies_1 = []
    accuracies_3   = []
    index = 0
    for train, test in kf.split(df):
        #separação conj de teste atributos e classe
        attr, df_class = attr_class(df)

        # 1-NN
        knn  = KNN(1)
        knn.fit(attr, df_class)
        predictions_simples = knn.predict(attr.iloc[test])
        acc_simples = (np.sum(predictions_simples == df_class.iloc[test]) / len(test)) * 100
        accuracies_1.append(acc_simples)

        # 3-NN
        knn  = KNN(3)
        knn.fit(attr, df_class)
        predictions_simples = knn.predict(attr.iloc[test])
        acc_simples = (np.sum(predictions_simples == df_class.iloc[test]) / len(test)) * 100
        accuracies_3.append(acc_simples)

    return accuracies_1, accuracies_3

accuracies_1, accuracies_3 = run(kc1, 'KC1')


In [13]:
def run(df, title, accuracies_1, accuracies_3):
    kf = KFold(n_splits=5)
    results = []
    n_proto = [10, 50, 100]
    for k in [1,3]:
        for n in n_proto:
            accuracies_simples = []
            accuracies_lvq1   = []
            accuracies_lvq2   = []
            accuracies_lvq3   = []
            index = 0
            for train, test in kf.split(df):
                #separação conj de teste atributos e classe
                attr, df_class = attr_class(df)

                # KNN com conjunto normal
                accuracies_simples = accuracies_1 if k == 1 else accuracies_3
                
                #definição do dataset de LVQ1
                lvq1 = LVQ1(n)
                aux = df.iloc[train]
                aux.reset_index(drop=True, inplace=True)
                res  = lvq1.trainPrototypes(aux)
                lvq1_df  = pd.DataFrame(res, columns = df.columns)
                
                #separação do treinamento LVQ1 atributos e classe
                X, Y = attr_class(lvq1_df)
                
                #KNN com conjunto do LVQ1
                knn  = KNN(k)
                knn.fit(X, Y)
                predictions_lvq1 = knn.predict(attr.iloc[test])
                acc_lvq1 = (np.sum(predictions_lvq1 == df_class.iloc[test]) / len(test)) * 100
                accuracies_lvq1.append(acc_lvq1)
                
                #definição do dataset de LVQ2.1
                lvq2 = LVQ2_1(n, 0.2)
                res  = lvq2.trainPrototypes(aux,lvq1_df)
                lvq2_df  = pd.DataFrame(res, columns = df.columns)
                
                #separação do treinamento LVQ2.1 atributos e classe
                X, Y = attr_class(lvq2_df)
                
                #KNN com conjunto do LVQ2.1
                knn  = KNN(k)
                knn.fit(X, Y)
                predictions_lvq2 = knn.predict(attr.iloc[test])
                acc_lvq2 = (np.sum(predictions_lvq2 == df_class.iloc[test]) / len(test)) * 100
                accuracies_lvq2.append(acc_lvq2)
                
                #definição do dataset de LVQ3
                lvq3 = LVQ3(n, 0.2)
                res  = lvq3.trainPrototypes(aux,lvq1_df)
                lvq3_df  = pd.DataFrame(res, columns = df.columns)
                
                #separação do treinamento LVQ3 atributos e classe
                X, Y = attr_class(lvq3_df)
                
                #KNN com conjunto do LVQ3
                knn  = KNN(k)
                knn.fit(X, Y)
                predictions_lvq3 = knn.predict(attr.iloc[test])
                acc_lvq3 = (np.sum(predictions_lvq3 == df_class.iloc[test]) / len(test)) * 100
                accuracies_lvq3.append(acc_lvq3)
            
            print("({}, {}): simples {} lvq1 {} lvq2 {} lvq3 {}".format(k, n, np.mean(accuracies_simples),np.mean(accuracies_lvq1),np.mean(accuracies_lvq2), np.mean(accuracies_lvq3)))
            temp = {
                'dataframe': title,
                '(k, prototypes)':(k, n),
                'acc_simples': np.mean(accuracies_simples),
                'std_simples': np.std(accuracies_simples),
                'acc_LVQ1'   : np.mean(accuracies_lvq1),
                'std_LVQ1'   : np.std(accuracies_lvq1),
                'acc_LVQ2.1' : np.mean(accuracies_lvq2),
                'std_LVQ2.1' : np.std(accuracies_lvq2),
                'acc_LVQ3'   : np.mean(accuracies_lvq3),
                'std_LVQ3'   : np.std(accuracies_lvq3),
            }
            results.append(temp)
    df = pd.DataFrame(results)
    return df

df = run(kc1, 'KC1', accuracies_1, accuracies_3)
df

(1, 10): simples 96.25491101079578 lvq1 84.4941518163704 lvq2 84.4941518163704 lvq3 84.44664587812812
(1, 50): simples 96.25491101079578 lvq1 82.7863020792291 lvq2 82.7863020792291 lvq3 83.02315633056028
(1, 100): simples 96.25491101079578 lvq1 79.5162724724477 lvq2 79.5162724724477 lvq3 80.98647994506422
(3, 10): simples 90.61149823822765 lvq1 84.54165775461267 lvq2 84.54165775461267 lvq3 84.54165775461267
(3, 50): simples 90.61149823822765 lvq1 84.49426438968378 lvq2 84.49426438968378 lvq3 84.44698359806823
(3, 100): simples 90.61149823822765 lvq1 83.92486857065664 lvq2 83.92486857065664 lvq3 84.11500489693913


,dataframe,"(k, prototypes)",acc_simples,std_simples,acc_LVQ1,std_LVQ1,acc_LVQ2.1,std_LVQ2.1,acc_LVQ3,std_LVQ3
0,KC1,"(1, 10)",96.254911,1.191967,84.494152,1.556999,84.494152,1.556999,84.446646,1.614778
1,KC1,"(1, 50)",96.254911,1.191967,82.786302,2.349853,82.786302,2.349853,83.023156,3.065205
2,KC1,"(1, 100)",96.254911,1.191967,79.516272,0.359099,79.516272,0.359099,80.986480,0.923941
3,KC1,"(3, 10)",90.611498,1.371464,84.541658,1.503010,84.541658,1.503010,84.541658,1.503010
4,KC1,"(3, 50)",90.611498,1.371464,84.494264,1.612353,84.494264,1.612353,84.446984,1.617788
5,KC1,"(3, 100)",90.611498,1.371464,83.924869,2.029098,83.924869,2.029098,84.115005,1.786248


## Plot


In [ ]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots


In [ ]:
def _printComparative(df):
    

    fig = make_subplots(rows=1, cols=1,
    specs=[[{"type": "Scatter"}]],
    subplot_titles=("Acurácia"))

    
    fig.add_trace(go.Scatter(x=df.index, y = df['acc_simples'], name='Simples', mode = 'lines+markers'), col = 1, row = 1) 
    fig.add_trace(go.Scatter(x=df.index, y = df['acc_LVQ1'], name='LVQ1', mode = 'lines+markers'), col = 1, row = 1)
    fig.add_trace(go.Scatter(x=df.index, y = df['acc_LVQ2.1'], name='LVQ2.1', mode = 'lines+markers'), col = 1, row = 1)
    fig.add_trace(go.Scatter(x=df.index, y = df['acc_LVQ3'], name='LVQ3', mode = 'lines+markers'), col = 1, row = 1)
    
    fig.update_layout(height=1000, width=1000)

    return fig


In [ ]:
fig = _printComparative(df)
fig.show()